# Laboratorio 6 Parte 2

### Reducción de dimensión: PCA y LDA

### 2019-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co

## Estudiantes

#### Primer Integrante: Kevin Martínez Gallego
#### Segundo Integrante: Andrés Mauricio Álvarez Ortiz

## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

## Indicaciones

Este ejercicio tiene como objetivo implementar varias técnicas de extracción de características (PCA y LDA) y usar SVM para resolver un problema de clasificación multietiqueta o multiclase.





In [6]:
import warnings
warnings.filterwarnings("ignore")


import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import scale

from numpy import round

import time


Para el problema de clasificación usaremos la siguiente base de datos: https://archive.ics.uci.edu/ml/datasets/Cardiotocography



Analice la base de datos, sus características, su variable de salida y el contexto del problema.



Analice y comprenda la siguiente celda de código donde se importan las librerías a usar y se carga la base de datos.

In [11]:
#cargamos la bd de entrenamiento
db = np.loadtxt('DB/DB_Fetal_Cardiotocograms.txt',delimiter='\t')  # Assuming tab-delimiter

X = db[:,0:22]

#Solo para dar formato a algunas variables
for i in range(1,7):
    X[:,i] = X[:,i]*1000

X = X
Y = db[:,22]

#Para darle formato de entero a la variable de salida

Y_l = []
for i in Y:
    Y_l.append(int(i))
Y = np.asarray(Y_l)
Y = np.expand_dims(Y,1)
print ("Dimensiones de la base de datos de entrenamiento. dim de X: " + str(np.shape(X)) + "\tdim de Y: " + str(np.shape(Y)))
print(np.unique(Y, return_counts = True))

Dimensiones de la base de datos de entrenamiento. dim de X: (2126, 22)	dim de Y: (2126, 1)
(array([1, 2, 3]), array([1655,  295,  176], dtype=int64))


In [12]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

## Ejercicio 1: Entrenamiento sin extracción de características

En la siguiente celda de código no tiene que completar nada. Analice, comprenda y ejecute el código y tenga en cuenta los resultados para completar la tabla que se le pide más abajo.

In [13]:
def executeSVM():
    #Para calcular el costo computacional
    tiempo_i = time.time()

    #Creamos el clasificador SVM. Tenga en cuenta que el problema es multiclase. 
    clf = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)

    #Implemetamos la metodología de validación

    Errores = np.ones(10)
    j = 0
    kf = KFold(n_splits=10)

    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entran y se valida el modelo sin hacer selección de características

        ######

        # Entrenamiento el modelo.
        model = clf.fit(X_train,y_train)

        # Validación del modelo
        ypred = model.predict(X_test)

        #######

        Errores[j] = classification_error(ypred, y_test)
        j+=1

    error = round(np.mean(Errores),4)
    std = round(np.std(Errores),4)
    execTime = round(time.time()-tiempo_i,4)
        
    print("\nError de validación sin aplicar extracción: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

    print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")
    
    return(error, std, execTime)

    #print str(ypred)
    #print str(y_test) 



1.1 Cuando se aplica PCA ¿es necesario estandarizar los datos? Si, No y por qué? En qué consiste dicha estandarización?

R/: Si, es necesario estándarizar o normalizar los datos. Este procedimiento se realiza con el objetivo de ajustar los datos para el correcto cálculo de las proyecciones de PCA, el cual requiere que los elementos de la matriz X tengan media cero.
    
1.2 La proyección de los datos que realiza PCA busca optimizar un medida, ¿Cuál? Explique.

R/: Inicialmente, la medida de información en PCA está relacionada al nivel de variación de las variables; se busca encontrar una transformación que entrega la mayor variación. Por otro lado, para conservar la estructura original de los datos es necesario minimizar las distancias entre los puntos (muestras) y sus respectivas proyecciones.

## Ejercicio 2: Entrenamiento con extracción de características

En la siguiente celda, complete el código donde le sea indicado. Consulte la documentación oficial de la librería mlxtend para los métodos de extracción de características, en este caso para PCA está en este [link](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).

En este ejercicio hará uso del método <font color='blue'>extract_features</font>  que recibe como parámetro el tipo de extracción si es PCA o LDA y el número de componentes o discriminantes, una vez se haga la transformación de las carácteristicas, se continua con el procedimiento del modelo a entrenar, en este caso es SVM.

In [14]:
'''
Feature Extraction Function
#Recibe 2 parámetros: 
1. el tipo de método de extracción (pca o lda como string),
2. el número componentes (para pca) o el número de discriminantes (para lda)

#Para este laboratorio solo se le pedirá trabajar con PCA, LDA es opcional.
'''

def extract_features(tipo, n):
    
    if tipo == 'pca':
    
        ext = PCA(n_components=n)
    
        return ext

    elif tipo == 'lda':
        
        ext = LDA(n_components=n)
        
        return ext
    
    else:
        print ("Ingrese un método válido (pca o lda)\n")

In [17]:
#Estandarizamos los datos
X = scale(X)

def executePCA(method, n_components):
    
    #Creamos el clasificador SVM. Tenga en cuenta que el problema es multiclase. 
    clf = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)
    
    #Para calcular el costo computacional
    tiempo_i = time.time()

    #Implemetamos la metodología de validación cross validation con 10 folds

    Errores = np.ones(10)
    j = 0
    kf = KFold(n_splits=10)

    for train_index, test_index in kf.split(X):

        #Aquí se aplica la extracción de características por PCA
        #Complete el código

        #Complete el código llamando el método extract_features. Tenga en cuenta lo que le pide el ejercicio 3.1
        ex = extract_features(method, n_components)

        #Fit de PCA
        ex.fit(X)#Complete el código con el fit correspondiente

        #Transforme las variables y genere el nuevo espacio de características de menor dimensión
        X_ex = ex.transform(X) #complete el código aquí para hacer la transformación

        #Se aplica CV-10

        X_train, X_test = X_ex[train_index], X_ex[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entrena y se valida el modelo luego de aplicar extracción de características con PCA o LDA

        ######

        # Entrenamiento el modelo.
        model = clf.fit(X_train,y_train)

        # Validación del modelo
        ypred = model.predict(X_test)

        #######

        Errores[j] = classification_error(ypred, y_test)
        j+=1

        
    error = round(np.mean(Errores),4)
    std = round(np.std(Errores),4)
    execTime = round(time.time()-tiempo_i,4)

    print("\nError de validación aplicando extracción: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

    print("\nEficiencia en validación aplicando extracción: " + str((1-np.mean(Errores))*100) + "%" )

    print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")
    
    return(error, std, execTime)


## Ejercicio 3

3.1 En la celda de código anterior, varíe los parámetros correspondientes al número de componentes principales a tener en cuenta (use 2, 10, 19 y 21 componentes principales) para PCA y complete la siguiente tabla de resultados:

In [21]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['SVM sin extracción','SVM + PCA','SVM + PCA','SVM + PCA','SVM + PCA']),
    '# de componentes' : pd.Series(['N/A',2,10,19,21]),
   })
df_types["Error de validación"] = ""
df_types["IC(std)"] = ""
df_types["Eficiencia"] = ""
df_types["Tiempo de ejecución"] = ""

df_types.set_index(['Tecnica','# de componentes'], inplace=True)

index = 0

err, std, executionTime = executeSVM()
df_types["Error de validación"][index] = err
df_types["IC(std)"][index] = std
df_types["Eficiencia"][index] = 1 - err
df_types["Tiempo de ejecución"][index] = executionTime
index += 1

n_components = [2, 10, 19, 21]

for i in range(np.size(n_components)):
    err, std, executionTime = executePCA('pca', n_components[i])
    df_types["Error de validación"][index] = err
    df_types["IC(std)"][index] = std
    df_types["Eficiencia"][index] = 1 - err
    df_types["Tiempo de ejecución"][index] = executionTime
    index += 1

#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget


Error de validación sin aplicar extracción: 0.07336345114713438 +/- 0.043533331744924374


Tiempo total de ejecución: 0.6435832977294922 segundos.

Error de validación aplicando extracción: 0.21483745238727964 +/- 0.1709424253642628

Eficiencia en validación aplicando extracción: 78.51625476127204%


Tiempo total de ejecución: 1.369903802871704 segundos.

Error de validación aplicando extracción: 0.08932589246168837 +/- 0.06387058313574974

Eficiencia en validación aplicando extracción: 91.06741075383115%


Tiempo total de ejecución: 0.7350757122039795 segundos.

Error de validación aplicando extracción: 0.07383514926034193 +/- 0.04357915071494415

Eficiencia en validación aplicando extracción: 92.6164850739658%


Tiempo total de ejecución: 0.7053482532501221 segundos.

Error de validación aplicando extracción: 0.07336345114713438 +/- 0.043533331744924374

Eficiencia en validación aplicando extracción: 92.66365488528656%


Tiempo total de ejecución: 0.8551223278045654 segundos.


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [22]:
qgrid_widget.get_changed_df()

Error de validación IC(std) Eficiencia  \
Tecnica            # de componentes                                          
SVM sin extracción N/A                           0.0734  0.0435     0.9266   
SVM + PCA          2                             0.2148  0.1709     0.7852   
                   10                            0.0893  0.0639     0.9107   
                   19                            0.0738  0.0436     0.9262   
                   21                            0.0734  0.0435     0.9266   

                                    Tiempo de ejecución  
Tecnica            # de componentes                      
SVM sin extracción N/A                           0.6436  
SVM + PCA          2                             1.3648  
                   10                            0.7351  
                   19                            0.7003  
                   21                            0.8501

3.2 Analizando los resultados del punto anterior que puede decir de la viabilidad de aplicar PCA para hacer reducción de dimensión en este problema?

R/: En este problema no evidenciamos una mejora significativa en el rendimiento del modelo al aplicar PCA.


3.3 Explique en sus palabras la principal ventaja que tiene LDA sobre PCA para resolver problemas de clasificación.

R/: PCA es un método no supervisado que NO nos garantiza encontrar una transformación que mantenga la capacidad discriminante entre clases, debido a que se centra en maximizar la variabilidad de los datos. Sin embargo, LDA sí fue pensado exclusivamente como un método supervisado para abordar problemas de clasificación, tal que su objetivo es justamente mantener la capacidad discriminante de los datos.

3.3 Explique en sus palabras las diferencias que existen entre los métodos de selección de características y los métodos de extracción de características vistos en el curso.

R/: La selección de características se enfoca en encontrar un subconjunto de las características originales, mientras que la extracción de características se enfoca en construir un conjunto nuevo de variables (mediante combinaciones lineales) a partir de las características originales.

## * Ejercicio 4: LDA (Punto opcional)

En este punto hará uso del método <font color='blue'>extract_features</font>  que recibe como parámetro el tipo de extracción si es PCA o LDA y el número de discriminantes, una vez se haga la transformación de las carácteristicas, se continua con el procedimiento del modelo a entrenar, en este caso es SVM. Esta es documentación para LDA [link ](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html)



In [ ]:
#Para calcular el costo computacional
tiempo_i = time.time()

#Estandarizamos los datos
X = scale(X)

#Implemetamos la metodología de validación cross validation con 10 folds

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    
    #Aquí se aplica la extracción de características por LDA
    
    #OPCIONAL

    ex = #Complete el código llamando el método extract_features.

    #Fit de LDA
    ex =#Complete el código con el fit correspondiente
    
    #Transforme las variables y genere el nuevo espacio de características de menor dimensión
    X_ex =   #complete el código aquí para hacer la transformación
    
    
    #Se aplica CV-10
    
    X_train, X_test = X_ex[train_index], X_ex[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  
   
    #Aquí se entrena y se valida el modelo luego de aplicar extracción de características con PCA o LDA
    
    ######
    
    # Entrenamiento el modelo.
    model = clf.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)
    
    #######

    Errores[j] = classification_error(ypred, y_test)
    j+=1
        

print("\nError de validación aplicando extracción: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

print("\nEficiencia en validación aplicando extracción: " + str((1-np.mean(Errores))*100) + "%" )

print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")


In [3]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['SVM sin extracción','SVM + LDA','SVM + LDA','SVM + LDA','SVM + LDA']),
    '# de discriminantes' : pd.Series(['N/A',2,10,19,21]),
   })
df_types["Error de validación"] = ""
df_types["IC(std)"] = ""
df_types["Eficiencia"] = ""
df_types["Tiempo de ejecución"] = ""

df_types.set_index(['Tecnica','# de discriminantes'], inplace=True)

#df_types.sort_index(inplace=True)
qgrid_widget2 = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget2

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [4]:
qgrid_widget2.get_changed_df()

Error de validación IC(std) Eficiencia  \
Tecnica            # de discriminantes                                          
SVM sin extracción N/A                                                          
SVM + LDA          2                                                            
                   10                                                           
                   19                                                           
                   21                                                           

                                       Tiempo de ejecución  
Tecnica            # de discriminantes                      
SVM sin extracción N/A                                      
SVM + LDA          2                                        
                   10                                       
                   19                                       
                   21